In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

2021.07.06

进行MPAS的后处理部分，计算每日气温的方差，并保存到同目录下的 output_nc 文件夹中，准备后续的NCL绘图

In [2]:
dir_in = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/mask_t2m/mask_res/"
filename_cn051  = "mask_sel_CN05.1_Tm_1961_2018_daily_025x025.nc"
filename_vr     = "mask_mean_t2m_98-17_VR.nc"
filename_rcm    = "mask_mean_t2m_98-17_RCM.nc"

ds_or = {}
ds_or['cn051']  = xr.open_dataset(dir_in + filename_cn051)
ds_or['vr']     = xr.open_dataset(dir_in + filename_vr)
ds_or['rcm']    = xr.open_dataset(dir_in + filename_rcm  )
# 提取变量
var = {}
var['cn051'] = ds_or['cn051']['tm']
var['vr'] = ds_or['vr']['t2m'] - 273.15
var['rcm'] = ds_or['rcm']['t2m'] - 273.15
# change coords
var_list = ['cn051', 'vr', 'rcm']
for i in var_list:
    rename_dict = dict(zip(var[i].coords.keys(), var['cn051'].coords.keys()))
    # show converting coords
    for rename_i in rename_dict:
        print(rename_i + " -----converting to----- " + rename_dict[rename_i])

    var[i] = var[i].rename(rename_dict)
    var[i]._coords = var['cn051']._coords
    var[i] = var[i].rename(i)


time -----converting to----- time
lon -----converting to----- lon
lat -----converting to----- lat
Time -----converting to----- time
longitude -----converting to----- lon
latitude -----converting to----- lat
Time -----converting to----- time
longitude -----converting to----- lon
latitude -----converting to----- lat


## 时间拆分出来之后计算98-17整个序列上的variance
（AM,JJA包含了季节循环，大概为主？）

In [5]:
# 将var拆分到am和jja
var_am  = {}
var_jja = {}
for ivar in var:
    var_am[ivar]  = var[ivar].sel(time = var[ivar].time.dt.month.isin([4,5]))
    var_jja[ivar] = var[ivar].sel(time = var[ivar].time.dt.month.isin([6,7,8]))
# 计算方差 dim[0] 即是时间维
variance_am  = {}
variance_jja = {}
for ind, ivar in enumerate(var_list):
    variance_am[ivar]  = var_am[ivar].var(dim=var_am[ivar].dims[0])
    variance_jja[ivar] = var_jja[ivar].var(dim=var_jja[ivar].dims[0])

/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 

## 计算AM,JJA的平均值，计算年际上的variance

In [5]:
# 将var拆分到am和jja
var_am  = {}
var_jja = {}
for ivar in var:
    var_am[ivar]  = var[ivar].sel(time = var[ivar].time.dt.month.isin([4,5]))
    var_jja[ivar] = var[ivar].sel(time = var[ivar].time.dt.month.isin([6,7,8]))
# 计算方差 dim[0] 即是时间维
variance_am  = {}
variance_jja = {}
for ind, ivar in enumerate(var_list):
    variance_am[ivar]  = var_am[ivar].groupby(var_am[ivar].time.dt.year).mean().var(dim='year')
    variance_jja[ivar] = var_jja[ivar].groupby(var_jja[ivar].time.dt.year).mean().var(dim='year')

/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 

## 计算ydaymean之后的结果 seasonal

In [3]:
# 将var拆分到am和jja
var_am  = {}
var_jja = {}
for ivar in var:
    var_am[ivar]  = var[ivar].sel(time = var[ivar].time.dt.month.isin([4,5]))
    var_jja[ivar] = var[ivar].sel(time = var[ivar].time.dt.month.isin([6,7,8]))
# 计算方差 dim[0] 即是时间维
variance_am  = {}
variance_jja = {}
for ind, ivar in enumerate(var_list):
    variance_am[ivar]   = var_am[ivar].groupby(var_am[ivar].time.dt.dayofyear).mean().var(dim='dayofyear')
    variance_jja[ivar]  = var_jja[ivar].groupby(var_jja[ivar].time.dt.dayofyear).mean().var(dim='dayofyear')

/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 

# 输出临时文件

In [4]:
# 合并字典中的计算结果到 ds_variance 输出到临时文件，准备后续NCL出图
ds_am_variance  = xr.merge(list(variance_am.values()) )
ds_jja_variance = xr.merge(list(variance_jja.values()) )
ds_am_variance.fillna(-1e30)
ds_jja_variance.fillna(-1e30)

dir_out = "/m2data2/yycheng/playground/MPAS_case_study/92-25km_vr_rcm_postprocess/visualization/check_obs/settled/t2m/space/output_nc/"
# 添加每个变量的fillvalue
encoding = {}
for i in var_list:
    encoding[i] = {"_FillValue":-1e30}

output_name = "annual"
ds_am_variance.to_netcdf(path = dir_out + "am_pre_variance_"+output_name+".nc", encoding = encoding)
ds_jja_variance.to_netcdf(path = dir_out + "jja_pre_variance_"+output_name+".nc", encoding = encoding)